## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
from fastai.models.cifar10.wideresnet import wrn_22
torch.backends.cudnn.benchmark = True
PATH = Path("data/cifar10/")
os.makedirs(PATH,exist_ok=True)

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

bs=512
sz=32
workers=7

In [4]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

def pad(img, p=4, padding_mode='reflect'):
        return Image.fromarray(np.pad(np.asarray(img), ((p, p), (p, p), (0, 0)), padding_mode))
def to_pil(img): return Image.fromarray(img)

def torch_tfms(size, conv_pil=False, to_numpy=False):
    mean,std=[0.4914 , 0.48216, 0.44653], [0.24703, 0.24349, 0.26159]
    normalize = transforms.Normalize(mean=mean, std=std)
    tfms = [transforms.ToTensor(), normalize]
    
    # Torch transforms with fastai dl
    if to_numpy: tfms = [np.array, Normalize(mean,std), lambda x: x[0].T]
        
    aug_tfms = [
        pad, # TODO: use `padding` rather than assuming 4
        transforms.RandomCrop(size),
        transforms.ColorJitter(.25,.25,.25),
        transforms.RandomRotation(2),
        transforms.RandomHorizontalFlip(),
    ]
    scale_size = 40
    padding = int((scale_size - size) / 2)
    
    train_tfms = transforms.Compose(aug_tfms + tfms)
    train_tfms.sz = size
    val_tfms = transforms.Compose(tfms)
    val_tfms.sz = size
    if conv_pil:
        train_tfms.transforms.insert(0, to_pil)
        val_tfms.transforms.insert(0, to_pil)
    return train_tfms, val_tfms
    
def torch_ds(data_path, tfms):
    train_tfms, val_tfms = tfms
    
    # Data loading code
    traindir = os.path.join(data_path, 'train')
    valdir = os.path.join(data_path, 'test')

    train_dataset = datasets.ImageFolder(traindir, train_tfms)
    val_dataset = datasets.ImageFolder(valdir, val_tfms)
    return train_dataset, val_dataset

def torch_loader(data_path, datasets):
    train_dataset, val_dataset = datasets
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
        num_workers=workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=bs*2, shuffle=False,
        num_workers=workers, pin_memory=True)

    data = ModelData(data_path, train_loader, val_loader)
    return data


In [5]:
def get_learner(data):
    m = wrn_22()
    learn = ConvLearner.from_model_data(m, data)
    learn.crit = nn.CrossEntropyLoss()
    learn.metrics = [accuracy]
    learn.half()
    return learn

### Torch Data Loader

In [6]:
tfms = torch_tfms(sz)
dsets = torch_ds(PATH, tfms)
torch_data = torch_loader(str(PATH), dsets)

In [7]:
learn = get_learner(torch_data)

In [8]:
# Warmup session
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                 
    0      2.313565   2.290653   0.1145    

CPU times: user 10.1 s, sys: 4.82 s, total: 14.9 s
Wall time: 18.9 s


[array([2.29065]), 0.1145]

In [9]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.297995   2.298381   0.1049    
    1      2.285433   2.269319   0.1227                   

CPU times: user 19.1 s, sys: 8.08 s, total: 27.2 s
Wall time: 28.6 s


[array([2.26932]), 0.12269999998211861]

### Fastai Data Loader

In [10]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [11]:
learn = get_learner(fastai_data)

In [12]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.191434   3.005444   0.0748    
    1      2.144863   2.142466   0.1894                   

CPU times: user 1min 58s, sys: 3min 8s, total: 5min 6s
Wall time: 50.2 s


[array([2.14247]), 0.1894]

### Fastai Default Collate

In [13]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_dc_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [14]:
# Hack to get the hidden collate function out of the torch dataloader class. 
collate_fn = torch.utils.data.DataLoader(None).collate_fn

In [15]:
fastai_dc_data.trn_dl.collate_fn = collate_fn
fastai_dc_data.val_dl.collate_fn = collate_fn

In [16]:
learn = get_learner(fastai_dc_data)

In [17]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.183863   3.273847   0.1022    
    1      1.980297   1.922481   0.2369                   

CPU times: user 1min 54s, sys: 3min 13s, total: 5min 7s
Wall time: 50.9 s


[array([1.92248]), 0.23689999990463256]

### Fastai DataLoader - Torch Transforms

In [18]:
tfms = torch_tfms(sz, to_numpy=True)

In [19]:
fdl_tds_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [20]:
fdl_tds_data.trn_ds.open_fn = Image.open
fdl_tds_data.val_ds.open_fn = Image.open

In [21]:
learn = get_learner(fdl_tds_data)

In [22]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.31803    2.303622   0.1006    
    1      2.303087   2.301844   0.1016                   

CPU times: user 3min 4s, sys: 3min 35s, total: 6min 39s
Wall time: 2min 12s


[array([2.30184]), 0.1016]

### Torch DataLoader - Fastai Datasets

In [23]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fdl = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)
tdl_fds = fdl.trn_ds, fdl.val_ds
torch_data = torch_loader(str(PATH), tdl_fds)

In [24]:
learn = get_learner(torch_data)

In [25]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.076291   3.559953   0.1264    
    1      1.938344   1.903328   0.2448                   

CPU times: user 18.9 s, sys: 8.98 s, total: 27.9 s
Wall time: 28.7 s


[array([1.90333]), 0.2447999997138977]

### Fastai DataLoader - Torch Dataset - Same as Torch Transforms

In [ ]:
# from fastai.dataset import FilesArrayDataset, folder_source, read_dir

In [45]:
tfms = torch_tfms(sz, to_numpy=True)
dsets = torch_ds(PATH, tfms)

In [46]:
fdl_tds_data = ImageData(PATH, list(dsets)+[None,None,None,None], bs, num_workers=workers, classes=dsets[0].classes)

In [47]:
learn = get_learner(fdl_tds_data)

In [ ]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)